# Set Up Databases and Populating Tickers and Historical Stock Data

# 1. Setup & Imports

In [ ]:
import sys
import os
import sqlite3
from pathlib import Path
from datetime import datetime, timedelta

# Import paths from the config module
from src.config import BASE_DIR, DB_DIR

# Define BASE_DIR and adjust sys.path before importing
BASE_DIR = Path(os.getcwd()).parent  # Points to stat_656_autotrader/ from Notebooks/
sys.path.append(str(BASE_DIR))
print(f"Project root added to sys.path: {BASE_DIR}")

from src.db_schema import DATABASES
from src.etl.populate_tickers import recreate_database, populate_tickers
from src.etl.populate_prices import ensure_prices_table, populate_prices
from src.utils.db_utils import fetch_all_asset_metadata, fetch_all_asset_prices

ModuleNotFoundError: No module named 'src'

# 2. Initialize SQLite Databases

In [ ]:
DB_DIR.mkdir(exist_ok=True)

def setup_databases():
    """Initialize databases and tables based on defined schemas."""
    for db_name, schemas in DATABASES.items():
        db_path = DB_DIR / db_name

        try:
            with sqlite3.connect(db_path) as conn:
                conn.execute("PRAGMA foreign_keys = ON;")
                cursor = conn.cursor()
                for schema in schemas:
                    cursor.execute(schema)
                conn.commit()
                print(f"✅ Tables created/verified in '{db_name}'")
        except sqlite3.Error as e:
            print(f"❌ Error setting up '{db_name}': {e}")

# Run database setup
setup_databases()

✅ Tables created/verified in 'assets.db'
✅ Tables created/verified in 'portfolio_management.db'
✅ Tables created/verified in 'accounting.db'
✅ Tables created/verified in 'modeling.db'
✅ Tables created/verified in 'exogenous.db'
✅ Tables created/verified in 'db_change_log.db'


# 3. Populate Asset Metadata (Tickers)

In [3]:
print("\n🔄 Recreating and populating tickers database...")
recreate_database()
populate_tickers()
print("✅ Tickers populated successfully.")


🔄 Recreating and populating tickers database...
✅ Tickers populated successfully.


In [4]:
# View the asset metadata
asset_metadata_df = fetch_all_asset_metadata()
print("DataFrame shape:", asset_metadata_df.shape)
print("DataFrame columns:", asset_metadata_df.columns.tolist())
display(asset_metadata_df.head(20))

Connecting to database: D:\dev\stat_656_autotrader\databases\assets.db
Raw rows fetched: [(1, 'CCLDO', 'CareCloud, Inc. 8.75% Series B Cumulative Redeemable Perpetual Preferred Stock', 'NASDAQ', 'us_equity', 1, '2025-03-31', None, '2025-03-31 01:26:01'), (2, 'BHVN', 'Biohaven Ltd.', 'NYSE', 'us_equity', 1, '2025-03-31', None, '2025-03-31 01:26:01'), (3, 'ZUMZ', 'Zumiez Inc. Common Stock', 'NASDAQ', 'us_equity', 1, '2025-03-31', None, '2025-03-31 01:26:01'), (4, 'EMR', 'Emerson Electric Co.', 'NYSE', 'us_equity', 1, '2025-03-31', None, '2025-03-31 01:26:01'), (5, 'CCL', 'Carnival Corporation', 'NYSE', 'us_equity', 1, '2025-03-31', None, '2025-03-31 01:26:01')]
Column names from DB: ['asset_id', 'symbol', 'name', 'exchange', 'asset_type', 'is_active', 'date_added', 'date_removed', 'fetched_at']
DataFrame shape: (7462, 9)
DataFrame columns: ['asset_id', 'symbol', 'name', 'exchange', 'asset_type', 'is_active', 'date_added', 'date_removed', 'fetched_at']


,asset_id,symbol,name,exchange,asset_type,is_active,date_added,date_removed,fetched_at
0,1,CCLDO,"CareCloud, Inc. 8.75% Series B Cumulative Rede...",NASDAQ,us_equity,1,2025-03-31,None,2025-03-31 01:26:01
1,2,BHVN,Biohaven Ltd.,NYSE,us_equity,1,2025-03-31,None,2025-03-31 01:26:01
2,3,ZUMZ,Zumiez Inc. Common Stock,NASDAQ,us_equity,1,2025-03-31,None,2025-03-31 01:26:01
3,4,EMR,Emerson Electric Co.,NYSE,us_equity,1,2025-03-31,None,2025-03-31 01:26:01
4,5,CCL,Carnival Corporation,NYSE,us_equity,1,2025-03-31,None,2025-03-31 01:26:01
5,6,CCK,Crown Holdings Inc.,NYSE,us_equity,1,2025-03-31,None,2025-03-31 01:26:01
6,7,CCJ,Cameco Corporation,NYSE,us_equity,1,2025-03-31,None,2025-03-31 01:26:01
7,8,EMP,"Entergy Mississippi, LLC First Mortgage Bonds,...",NYSE,us_equity,1,2025-03-31,None,2025-03-31 01:26:01
8,9,ALDX,"Aldeyra Therapeutics, Inc. Common Stock",NASDAQ,us_equity,1,2025-03-31,None,2025-03-31 01:26:01
9,10,CCI,Crown Castle Inc.,NYSE,us_equity,1,2025-03-31,None,2025-03-31 01:26:01


NameError: name 'STOP' is not defined

# 4. Populate Historical Stock Prices

In [6]:
print("\n🔄 Ensuring prices table exists and populating historical prices...")
ensure_prices_table()
populate_prices()
print("✅ Historical stock prices populated successfully.")


🔄 Ensuring prices table exists and populating historical prices...
Connecting to database: D:\dev\stat_656_autotrader\databases\assets.db


Alpaca Download Progress: 100%|██████████| 7462/7462 [1:54:25<00:00,  1.09it/s]  



Fetched 10055017 rows of stock data in 6865.55 seconds.
Processed 7462 tickers, missing 2856 tickers, 2852 tickers did not have enough time.
Processing time per ticker: 0.92 seconds.


Inserting Prices into DB: 100%|██████████| 10055017/10055017 [14:56<00:00, 11216.68it/s]


✅ Historical stock prices populated successfully.


In [7]:
# Fetch and display the historical prices
print("\n📊 Fetching and displaying historical stock prices...")
prices_df = fetch_all_asset_prices()

# Display basic info and the first 20 rows
print(f"DataFrame shape: {prices_df.shape}")
print(f"Columns: {prices_df.columns.tolist()}")
display(prices_df.head(20))


📊 Fetching and displaying historical stock prices...
Connecting to database: D:\dev\stat_656_autotrader\databases\assets.db
DataFrame shape: (10055017, 11)
Columns: ['price_id', 'asset_id', 'symbol', 'date', 'open', 'high', 'low', 'close', 'adjusted_close', 'volume', 'fetched_at']


,price_id,asset_id,symbol,date,open,high,low,close,adjusted_close,volume,fetched_at
0,1988,3,ZUMZ,2016-01-04,14.80,15.4400,14.1941,15.1600,None,None,2025-03-31 03:21:10
1,4311,4,EMR,2016-01-04,47.32,47.3400,46.5200,47.3000,None,None,2025-03-31 03:21:10
2,6634,5,CCL,2016-01-04,54.15,54.2200,53.2200,54.2000,None,None,2025-03-31 03:21:10
3,8957,6,CCK,2016-01-04,49.78,49.8200,48.8000,49.7600,None,None,2025-03-31 03:21:10
4,11280,7,CCJ,2016-01-04,11.95,12.1010,11.7061,12.0500,None,None,2025-03-31 03:21:11
5,15748,9,ALDX,2016-01-04,6.59,6.7800,6.4600,6.6800,None,None,2025-03-31 03:21:11
6,18071,10,CCI,2016-01-04,85.63,85.9300,84.8300,85.3900,None,None,2025-03-31 03:21:11
7,20394,11,EMO,2016-01-04,12.31,12.9500,12.3100,12.8700,None,None,2025-03-31 03:21:11
8,24520,15,EMN,2016-01-04,66.50,66.6600,65.1200,66.6300,None,None,2025-03-31 03:21:12
9,26843,16,EML,2016-01-04,18.75,19.0390,18.4750,18.9795,None,None,2025-03-31 03:21:12
